# Using TenserFlow to solve it

In [551]:
import numpy as np
import pandas as pd
import sklearn
import tensorflow as tf
import matplotlib.pyplot as plt
from tensorflow import keras
import os
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from nltk.corpus import stopwords

# Check for TensorFlow GPU access
print(f"TensorFlow has access to the following devices:\n{tf.config.list_physical_devices()}")

# See TensorFlow version
print(f"TensorFlow version: {tf.__version__}")

TensorFlow has access to the following devices:
[PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU'), PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]
TensorFlow version: 2.8.0


read and clean Data

In [552]:
with open('list.txt','r') as st:
    list_stopWords = st.read()
list_stopWords = list_stopWords.split('\n')


def stopwords(text):
    '''
    FUNCATION
    drop the arabic stopping words values from the input
    and then return the return it
    INPUT
    text of str data type
    OUTPUT
    text of str data type
    '''
    text = ' '.join([word for word in text.split(' ') if word not in list_stopWords])
    text = text.replace("\n", "")
    text = text.replace("-", "")
    text = text.replace("--", "")
    text = text.replace("››", "")
    text = text.replace("‹‹", "")
    text = text.replace("‰", "")
    text = text.replace("••", "")
    text = text.replace("•", "")
    text = text.replace("’", "")
    text = text.replace("‘", "")
    text = text.replace("|", "")   
    text = text.replace(";", "")  
    text = text.replace("?", "")  
    text = text.replace("«", "")  
    text = text.replace("·", "")  
    text = text.replace("»", "")  
    text = text.replace("اء", "")  
    text = text.replace("ءيه", "")  
    text = text.replace("ءولا", "")  
    text = text.replace("ءري", "")  
    text = text.replace("ءذون", "")  
    text = text.replace("ءد", "")  

    return text
    
df = pd.read_csv('trainCopy.csv')
df.drop(['poet_name', 'Unnamed: 0'], axis=1, inplace = True) # drop useless columns
df.poem_text = df.poem_text# select only the first 2000 charecter in the poem_text
df.dropna(inplace=True) # drop the missings 
df.poem_text = df.poem_text.apply(lambda x:stopwords(x)) #remove the stopping words
df.poem_title = df.poem_title.apply(lambda x:stopwords(x)) #remove the stopping words

df.head()

,poem_id,poem_text,poem_title,poet_id,poet_cat_أفغانستان,poet_cat_إيران,poet_cat_الأردن,poet_cat_الإمارات,poet_cat_البحرين,poet_cat_الجزائر,...,poet_cat_تونس,poet_cat_سوريا,poet_cat_شعراء العراق والشام,poet_cat_عمان,poet_cat_فلسطين,poet_cat_قطر,poet_cat_لبنان,poet_cat_ليبيا,poet_cat_مصر,poet_cat_موريتانيا
0,21,عيناك غابتا نخيل ساعه السحر شرفتان يناي عنهما ...,انشوده المطر,2,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,65546,ازال يدي قدحي ياليل اين تفرق الشرب زلت اشربها ...,اقداح احلام,2,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,65561,مقلتيك ارتشفت النجوم وعانقت امالي الايبه وسابق...,هوي,2,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,66170,تزيديه لوعه يلقاك لينسي لديك اكتءابه قربي مقلت...,تزيديه لوعه,2,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,66171,عطرت احلامي الشذي شعرك المسترسل الاسود الجو حو...,عبير,2,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


Number of poets before filtering

In [553]:
len(np.unique(df.poet_id))

651

number of poets after filtering, so any poet should have at leat 15 poems

In [554]:
filteredDF = df.groupby('poet_id').filter(lambda x : len(x)>15)
len(np.unique(filteredDF.poet_id))

429

In [555]:
filteredDF.shape

(54930, 30)

In [556]:
# Shuffling a Pandas dataframe with sklearn
from sklearn.utils import shuffle
shuffled = shuffle(filteredDF).reset_index()
shuffled.head()

,index,poem_id,poem_text,poem_title,poet_id,poet_cat_أفغانستان,poet_cat_إيران,poet_cat_الأردن,poet_cat_الإمارات,poet_cat_البحرين,...,poet_cat_تونس,poet_cat_سوريا,poet_cat_شعراء العراق والشام,poet_cat_عمان,poet_cat_فلسطين,poet_cat_قطر,poet_cat_لبنان,poet_cat_ليبيا,poet_cat_مصر,poet_cat_موريتانيا
0,6267,3246,ستاتي الدهنا قصد مرجم تمطت بالفلاه ركابها قصاي...,ستاتي الدهنا قصد مرجم,118,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,52814,82853,جرحي صدر الشباب يقاتل الكل ينزف الغرام القاتل ...,شجويه,606,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,46424,73640,اشرق الحي حلت بساحته تاج الكرام قلبي علقا جميل...,اشرق الحي حلت بساحته,465,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
3,14611,62162,تهزهم مداحهم هز الكماه عوالي المران كانوا امتد...,تهزهم مداحهم,173,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,54110,83344,ربك يعتمد بالم يلج السما يوسف فمشيت ابيك طرق ا...,ربك يعتمد,645,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0


In [614]:
print(shuffled.shape)
dfSample = filteredDF[:]
print(dfSample.shape)

(54930, 31)
(54930, 30)


## Modling

In [621]:
vocab_size = 5000
embedding_dim = 128
max_length = 400
trunc_type = 'post'
padding_type = 'post'
oov_tok = '<OOV>'
training_portion = .80 # 80% train 20% val

In [622]:
from collections import Counter

# Count unique words
def counter_word(text_col):
    count = Counter()
    for text in text_col.values:
        for word in text.split():
            count[word] += 1
    return count


counter_text = counter_word(dfSample.poem_text)
counter_title = counter_word(dfSample.poem_title)

print('Number of uniqe Words is poems: {}'.format(len(counter_text)))
print('Number of uniqe Words is title poems: {}'.format(len(counter_title)))

print(counter_text.most_common(10))

# saving the number of unique words in variable to use it later
num_unique_words = len(counter_text)

vocab_size = num_unique_words

Number of uniqe Words is poems: 478566
Number of uniqe Words is title poems: 50570
[('اله', 19238), ('الناس', 8922), ('الارض', 8076), ('الدهر', 7878), ('اليل', 7799), ('قلبي', 7671), ('الهوي', 7651), ('الزمان', 6489), ('منك', 6417), ('الحب', 6356)]


Data splitting

In [623]:
# Split dataset into training and validation set
train_size = int(dfSample.shape[0] * training_portion)

train_df = dfSample[:train_size]
val_df = dfSample[train_size:]

# split text and labels
train_sentences = train_df.poem_text.to_numpy()
train_labels = train_df.poet_id.to_numpy()
val_sentences = val_df.poem_text.to_numpy()
val_labels = val_df.poet_id.to_numpy()

print(len(train_sentences))
print(len(train_labels))
print(len(val_sentences))
print(len(val_labels))

print(train_sentences.shape, val_sentences.shape)
(unique, counts) = np.unique(train_labels, return_counts=True)
frequencies = np.asarray((unique, counts)).T
print('there are {} of unique poets'.format(len(unique)))
print(frequencies)

(unique, counts) = np.unique(val_labels, return_counts=True)
frequencies = np.asarray((unique, counts)).T
print('there are {} of unique poets'.format(len(unique)))
print(frequencies)

numOfClasses = len(unique)+1

43944
43944
10986
10986
(43944,) (10986,)
there are 275 of unique poets
[[   2  181]
 [   7  581]
 [   9   35]
 [  10  142]
 [  15   18]
 [  19   22]
 [  23   27]
 [  25   78]
 [  31  272]
 [  32  128]
 [  33  123]
 [  34   23]
 [  35  274]
 [  36   67]
 [  67   41]
 [  68  376]
 [  71   27]
 [  91   27]
 [  92  348]
 [  93   16]
 [  97  309]
 [ 100   18]
 [ 101   16]
 [ 103  143]
 [ 104   75]
 [ 105   31]
 [ 106  190]
 [ 107   31]
 [ 108   46]
 [ 113   16]
 [ 116  277]
 [ 117 1587]
 [ 118  601]
 [ 119  959]
 [ 121   22]
 [ 122   55]
 [ 123   46]
 [ 125  150]
 [ 127  116]
 [ 129   45]
 [ 130   24]
 [ 131  164]
 [ 132  410]
 [ 134   54]
 [ 135  192]
 [ 139   38]
 [ 141   26]
 [ 142   77]
 [ 143   30]
 [ 144  156]
 [ 145  103]
 [ 146   24]
 [ 147   27]
 [ 148   26]
 [ 149   23]
 [ 150   24]
 [ 151   21]
 [ 152  285]
 [ 153   39]
 [ 154   54]
 [ 155   76]
 [ 156  152]
 [ 158  185]
 [ 159  285]
 [ 160   76]
 [ 161  130]
 [ 162  474]
 [ 163   57]
 [ 164  272]
 [ 165  118]
 [ 167  237]
 [ 16

### Tokenize
Tokenize is the process of lending each word a uniqe number to represent the text in numbercal way, we use the Maximum number of words is the unique words which we created the variable earlier

In [624]:
# Tokenize
from tensorflow.keras.preprocessing.text import Tokenizer

# vectorize a text corpus by turning each text into a sequence of integers
tokenizer = Tokenizer(num_words=num_unique_words)
tokenizer.fit_on_texts(train_sentences) # fit only to training
word_index = tokenizer.word_index
dict(list(word_index.items())[0:10])

{'اله': 1,
 'الناس': 2,
 'الدهر': 3,
 'الارض': 4,
 'اليل': 5,
 'الهوي': 6,
 'منك': 7,
 'قلبي': 8,
 'الزمان': 9,
 'تري': 10}

convert it to sequences to use it for both variables train and valiadation

In [625]:
train_sequences = tokenizer.texts_to_sequences(train_sentences)
validation_sequences = tokenizer.texts_to_sequences(val_sentences)

print(train_sequences[2])

[5025, 57058, 151, 32004, 3238, 217025, 156634, 1395, 536, 4409, 4059, 156635, 3552, 3139, 721, 29666, 4758, 4814, 26707, 26708, 16768, 15773, 945, 318, 802, 631, 263, 92068, 17130, 12678, 8437, 9266, 3261, 1182, 39952, 25094, 17131, 5301, 342, 842, 16430, 4759, 24390, 42042, 4866, 5301, 342, 1821, 4057, 886, 61570, 4867, 1414, 2596, 353, 5301, 342, 11874, 6580, 915, 6, 25095, 217026, 23697, 3888, 217027, 40, 49852, 5301, 342, 1362, 9, 687, 24, 2367, 2338, 6, 125743, 15486, 61571, 81636, 1070, 11875, 5645, 61572, 7048, 15191, 44, 8661, 42043, 9791, 13822, 4083, 470, 19729, 884, 217028, 92069, 25874, 25096, 1324, 9266, 58, 16082, 81637, 4148, 4408, 39953, 10833, 16769, 81638, 49853, 151, 12445, 156636, 125744, 94, 156637, 886, 112, 802, 11348, 61573, 886, 1394, 13823, 2205, 89, 10672, 46957, 217029, 3140, 156638, 1087, 11348, 12446, 4355, 2263, 2205, 46958, 2190, 156639, 156640, 57059, 1897, 217030, 3, 1714, 16770, 210, 8538, 1404, 8145, 11875, 816, 20767, 2728, 39954, 3261, 217031, 615

### Pad
we use the pad to make sure that all the text are having the same length so if the max length of words in 153 words the all column will contain the same and if there is no values will added zore Instead

In [626]:
train_padded = pad_sequences(train_sequences, maxlen=max_length, padding=padding_type, truncating=trunc_type)
validation_padded = pad_sequences(validation_sequences, maxlen=max_length, padding=padding_type, truncating=trunc_type)

print(len(validation_sequences))
print(validation_padded.shape)
print(len(train_sequences[0]))
print(len(train_padded[0]))

print(len(train_sequences[1]))
print(len(train_padded[1]))

print(len(train_sequences[2]))
print(len(train_padded[2]))

print(len(train_sequences[3]))
print(len(train_padded[3]))

10986
(10986, 400)
373
400
478
400
184
400
110
400


In [627]:
print(train_padded[2])


[  5025  57058    151  32004   3238 217025 156634   1395    536   4409
   4059 156635   3552   3139    721  29666   4758   4814  26707  26708
  16768  15773    945    318    802    631    263  92068  17130  12678
   8437   9266   3261   1182  39952  25094  17131   5301    342    842
  16430   4759  24390  42042   4866   5301    342   1821   4057    886
  61570   4867   1414   2596    353   5301    342  11874   6580    915
      6  25095 217026  23697   3888 217027     40  49852   5301    342
   1362      9    687     24   2367   2338      6 125743  15486  61571
  81636   1070  11875   5645  61572   7048  15191     44   8661  42043
   9791  13822   4083    470  19729    884 217028  92069  25874  25096
   1324   9266     58  16082  81637   4148   4408  39953  10833  16769
  81638  49853    151  12445 156636 125744     94 156637    886    112
    802  11348  61573    886   1394  13823   2205     89  10672  46957
 217029   3140 156638   1087  11348  12446   4355   2263   2205  46958
   219

Here you can see the differnt between the sentences, sequences and the padded

In [628]:
print(train_sentences[2], '\n')
print(train_sequences[2], '\n')
print(train_padded[2])

مقلتيك ارتشفت النجوم وعانقت امالي الايبه وسابقت جناح الخيال بروحي روحك الواثبه اطلت فكانت سنا ذبا بعينيك بسمه ذبه ردتها مناي اناشيد ضي القمر تغني ليالي الربيع فتحلم ازهاره بالمطر ويمضي صداها يهز الضي ويغفو الزورق المنتظر خذي الكاس بلي صداك العميق ارتج قاعها شراب خذي الكاس جف الرحيق صدي هامس القرار ليتني سقيت التراب خذي الكاس زرعت الكروم قبر الهوي الخاسر فاعراقها تستعيد الشراب وتشتفه يد العاصر خذي الكاس نسيت الزمان حياتي سوي حاضر انتظار الهوي جلوسي الشاطيء المقفر وارسال طرفي يجوب العباب ويرتد افقه الاسمر اهل الشراع الضحوك الامنيات انظري انكرت هواك الجوج وقلبي واشواقك العارمه وضلت وهده الكبري صداها فيا ظالمه تجنيت حسبت النعاس ذبولا الزهره النمه اتنسين التماع النجوم خطانا وانفاسنا الواجفه وكيف احتضنا صدي القلوب تغني القبله الراجفه صدي لج احتراق الشفاه زال غيهب العاطفه ورانت الاعين الوامقات ظلال القبله النيه تنادي رغبه الشفاه ويمنعها الشك والواشيه فترتج ضغطه اليدين جكعنا الدهر ثانيه شقيقه روحي تذكرين ند سيبقي يجوب السنين وهمس الانجم الحالمات يهز التماعاتها بالرنين تسل فجوه الستار اليك تذكر

check the result by decoding text

In [629]:
reverse_word_index = dict([(value, key) for (key, value) in word_index.items()])

def decode_article(text):
    return ' '.join([reverse_word_index.get(i, '?') for i in text])
print(decode_article(train_padded[2]))
print('---')
print(train_sentences[2])

مقلتيك ارتشفت النجوم وعانقت امالي الايبه وسابقت جناح الخيال بروحي روحك الواثبه اطلت فكانت سنا ذبا بعينيك بسمه ذبه ردتها مناي اناشيد ضي القمر تغني ليالي الربيع فتحلم ازهاره بالمطر ويمضي صداها يهز الضي ويغفو الزورق المنتظر خذي الكاس بلي صداك العميق ارتج قاعها شراب خذي الكاس جف الرحيق صدي هامس القرار ليتني سقيت التراب خذي الكاس زرعت الكروم قبر الهوي الخاسر فاعراقها تستعيد الشراب وتشتفه يد العاصر خذي الكاس نسيت الزمان حياتي سوي حاضر انتظار الهوي جلوسي الشاطيء المقفر وارسال طرفي يجوب العباب ويرتد افقه الاسمر اهل الشراع الضحوك الامنيات انظري انكرت هواك الجوج وقلبي واشواقك العارمه وضلت وهده الكبري صداها فيا ظالمه تجنيت حسبت النعاس ذبولا الزهره النمه اتنسين التماع النجوم خطانا وانفاسنا الواجفه وكيف احتضنا صدي القلوب تغني القبله الراجفه صدي لج احتراق الشفاه زال غيهب العاطفه ورانت الاعين الوامقات ظلال القبله النيه تنادي رغبه الشفاه ويمنعها الشك والواشيه فترتج ضغطه اليدين جكعنا الدهر ثانيه شقيقه روحي تذكرين ند سيبقي يجوب السنين وهمس الانجم الحالمات يهز التماعاتها بالرنين تسل فجوه الستار اليك تذكر

Defining model

In [630]:
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(embedding_dim)),
    tf.keras.layers.Dense(embedding_dim, activation='relu'),
    tf.keras.layers.Dense(numOfClasses, activation='softmax')
])

model.summary()

Model: "sequential_38"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_38 (Embedding)    (None, None, 128)         61256448  
                                                                 
 bidirectional_38 (Bidirecti  (None, 256)              263168    
 onal)                                                           
                                                                 
 dense_76 (Dense)            (None, 128)               32896     
                                                                 
 dense_77 (Dense)            (None, 156)               20124     
                                                                 
Total params: 61,572,636
Trainable params: 61,572,636
Non-trainable params: 0
_________________________________________________________________


Train and save it

In [612]:
checkpoint_path = "training_1/cp.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)

# Create a callback that saves the model's weights
cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_path,
                                                 save_weights_only=True,
                                                 verbose=1)

model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
num_epochs = 10
history = model.fit(train_padded, train_labels, epochs=num_epochs, validation_data=(validation_padded, val_labels), verbose=2, callbacks=[cp_callback])

Epoch 1/10


2022-03-01 00:24:58.529515: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2022-03-01 00:24:59.101971: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2022-03-01 00:24:59.199461: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2022-03-01 00:25:00.371059: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2022-03-01 00:25:00.389283: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2022-03-01 00:25:02.743050: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2022-03-01 00:25:02.977295: I tensorflow/core/grappler/optimizers/cust


Epoch 1: saving model to training_1/cp.ckpt
1/1 - 7s - loss: 0.6082 - accuracy: 0.0000e+00 - val_loss: 0.5185 - val_accuracy: 0.5000 - 7s/epoch - 7s/step
Epoch 2/10

Epoch 2: saving model to training_1/cp.ckpt
1/1 - 1s - loss: 0.5693 - accuracy: 0.5500 - val_loss: 0.4853 - val_accuracy: 0.5000 - 1s/epoch - 1s/step
Epoch 3/10

Epoch 3: saving model to training_1/cp.ckpt
1/1 - 1s - loss: 0.5328 - accuracy: 0.5500 - val_loss: 0.4459 - val_accuracy: 0.5000 - 921ms/epoch - 921ms/step
Epoch 4/10

Epoch 4: saving model to training_1/cp.ckpt
1/1 - 1s - loss: 0.4898 - accuracy: 0.5500 - val_loss: 0.3940 - val_accuracy: 0.5000 - 751ms/epoch - 751ms/step
Epoch 5/10

Epoch 5: saving model to training_1/cp.ckpt
1/1 - 1s - loss: 0.4327 - accuracy: 0.5500 - val_loss: 0.3226 - val_accuracy: 0.5000 - 718ms/epoch - 718ms/step
Epoch 6/10

Epoch 6: saving model to training_1/cp.ckpt
1/1 - 1s - loss: 0.3544 - accuracy: 0.5500 - val_loss: 0.2256 - val_accuracy: 0.5000 - 708ms/epoch - 708ms/step
Epoch 7/10


### Conclusion

The run takes so much time but the result was 67% accuracy and validation accuracy 72% which is great

Finlly apply it to the testing dataset to predict the results